In [ ]:
from modules.csv_processor import CSVProcessor
from modules.data_augmentation import DataAugmentation
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import csv

In [ ]:
processor = CSVProcessor("../Training Data")
X_train, y_train = processor.split()
#processor = CSVProcessor("../Testing Data")
#X_test, y_test = processor.split()
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [ ]:
data_augmentation = DataAugmentation(X_train, y_train)
X_train, y_train = data_augmentation.apply_data_augmentation()

In [ ]:
scaler = StandardScaler()

# Fit the scaler on the training features and transform them
X_train_scaled = scaler.fit_transform(X_train)

# Transform the test features using the fitted scaler
X_test_scaled = scaler.transform(X_test)

In [ ]:
# 4. Split the Data into Training and Test Sets
X_train, X_test = X_train_scaled, X_test_scaled

In [ ]:
# Test different hyperparameter combinations for the Keras model
"""
# List of hyperparameter combinations
input_shape = X_train.shape[1]
num_classes = len(np.unique(y_train))
param_combinations = [
    {'hidden_layers': 1, 'hidden_units': 64, 'dropout': 0.5, 'batch_size': 4, 'epochs': 100},
    {'hidden_layers': 2, 'hidden_units': 128, 'dropout': 0.5, 'batch_size': 8, 'epochs': 150},
    {'hidden_layers': 3, 'hidden_units': 256, 'dropout': 0.5, 'batch_size': 16, 'epochs': 200},
]

# Create an empty list to store the results
results = []

# Iterate over each parameter combination
for params in param_combinations:
    # Create the model with the given parameters
    model = Sequential()
    for _ in range(params['hidden_layers']):
        model.add(Dense(params['hidden_units'], activation='relu'))
        model.add(BatchNormalization())
        model.add(Dropout(params['dropout']))
    model.add(Dense(num_classes, activation='softmax'))
    
    # Compile and fit the model
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    model.fit(X_train, y_train, batch_size=params['batch_size'], epochs=params['epochs'], verbose=1)
    
    # Evaluate the model on the testing data
    loss, accuracy = model.evaluate(X_test, y_test)
    
    # Append the parameters and accuracy to the results list
    result = {
        'hidden_layers': params['hidden_layers'],
        'hidden_units': params['hidden_units'],
        'dropout': params['dropout'],
        'batch_size': params['batch_size'],
        'epochs': params['epochs'],
        'accuracy': accuracy
    }
    results.append(result)

# Save the results to a CSV file
filename = 'model_results.csv'
keys = results[0].keys()
with open(filename, 'w', newline='') as file:
    writer = csv.DictWriter(file, keys)
    writer.writeheader()
    writer.writerows(results)
"""

In [ ]:
input_shape = X_train.shape[1]
num_classes = len(np.unique(y_train))

model = Sequential([
    Dense(64, activation='relu', input_shape=(input_shape,)),
    BatchNormalization(),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])

In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Fit the model to the training data
model.fit(X_train, y_train, batch_size=4, epochs=100, verbose=1)

In [ ]:
# Evaluate the model on the testing data
loss, accuracy = model.evaluate(X_test, y_test)

print("Testing Accuracy:", accuracy)